In [ ]:
import numpy as np
from parcels import FieldSet, ParticleSet, JITParticle, ScipyParticle, AdvectionRK4, ErrorCode, Variable, ParcelsRandom,   DiffusionUniformKh, AdvectionDiffusionM1, AdvectionDiffusionEM
from datetime import timedelta
import math

In [2]:
# Particle set
"""
This sets up the oil particle with different variables used in the kernels.

The water fraction is the amount of water in the oil droplets from emulsification, and affects the density and the viscosity.
The evaporation fraction affects the viscosity of the oil.
"""

class OilParticle(JITParticle):
    viscosity = Variable('viscosity', dtype=np.float32, initial=0.053)
    density = Variable('density', dtype=np.float32, initial=902)
    mass = Variable('mass', dtype=np.float32, initial=902)
    original_mass = Variable('original_mass', dtype=np.float32, initial=902)
    original_density = Variable('original_density', dtype=np.float32, initial=902)
    water_fraction = Variable('water_fraction', dtype=np.float32, initial=0)
    max_water_fraction = Variable('max_water_fraction', dtype=np.float32, initial=0.9)
    fraction_evaporated = Variable('fraction_evaporated', dtype=np.float32, initial=0)
    is_emulsified = Variable('is_emulsified', dtype=np.float32, initial=0)
    emulsification_time = Variable('emulsification_time', dtype=np.float32, initial=0)
    vapor_pressure = Variable('vapor_pressure', dtype=np.float32, initial=28112.246876047448)
    interfacial_tension = Variable('interfacial_tension', dtype=np.float32, initial=0.03)
    area = Variable('area', dtype=np.float32, initial = 0.1)
    interfacial_area = Variable('interfacial_area', dtype=np.float32, initial = 0)
    d_min = Variable('d_min', dtype=np.float32, initial = 1e-6)
    d_max = Variable('d_min', dtype=np.float32, initial = 1e-5)
    max_diameter = Variable('max_diameter', dtype=np.float32, initial = 1e-4)
    min_diameter = Variable('min_diameter', dtype=np.float32, initial = 1e-6)
    rise_velocity = Variable('rise_velocity', dtype=np.float32, initial = 0)

NameError: name 'JITParticle' is not defined

In [ ]:
# Field set

fname = 'GlobCurrent_example_data/*.nc'
filenames = {'U': fname, 'V': fname}
variables = {'U': 'eastward_eulerian_current_velocity', 'V': 'northward_eulerian_current_velocity'}
dimensions = {'U': {'lat': 'lat', 'lon': 'lon', 'time': 'time'}, # In the GlobCurrent data the dimensions are also called 'lon', 'lat' and 'time'
              'V': {'lat': 'lat', 'lon': 'lon', 'time': 'time'}}
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

fieldset.add_constant('density_water', 1028)
fieldset.add_constant('viscosity_water', 0.00122/1028)
fieldset.add_constant('g', 9.81)
fieldset.add_constant('wind_threshold', 5)
fieldset.add_constant('wind_speed', 6)
fieldset.add_constant('sea_surface_temperature', 10)
fieldset.add_constant('mld', 50)
fieldset.add_constant('density_air', 1.22)
fieldset.add_constant('wave_age', 35)
fieldset.add_constant('surface_z', 0)
fieldset.add_constant('v_k', 0.4)
fieldset.add_constant('theta', 1)
fieldset.add_constant('phi', 0.9)

### Kernels

In [ ]:
# Evaporation

def Evaporation(particle, fieldset, time):
    
    if particle.depth == 0:
    
        # Mass Transport Coefficient, K
        if abs(fieldset.wind_speed) < 10:
            K = 0.0025 * wind_speed**0.78
        else:
            K = 0.06 * wind_speed**2

        decay = -1 * particle.area * K * particle.vapor_pressure / (8.314 * fieldset.sea_water_temperature * particle.mass)
        particle.mass += particle.dt * (particle.original_mass * decay * math.exp(decay * particle.emulsification_time))
        particle.emulsification_time += particle.dt

        particle.fraction_evaporated = (particle.original_mass - particle.mass) / particle.original_mass

In [ ]:
# Emulsification

def Emulsification(particle, fieldset, time):
    
    if particle.depth == 0:
    
        max_interfacial_area = (6 / particle.d_min) * (particle.max_water_fraction / (1 - particle.max_water_fraction))

        k_emul = fieldset.wind_speed**2 * (6 * 2.02e-6)/(1e-5)

        particle.interfacial_area += particle.dt * (k_emul * (1 - (particle.interfacial_area/max_interfacial_area)))
        if particle.interfacial_area > max_interfacial_area:
            particle.interfacial_area = max_interfacial_area

        particle.water_fraction = particle.interfacial_area * particle.d_max / (6 + (particle.interfacial_area * particle.d_max))
        if particle.water_fraction > particle.max_water_fraction:
            particle.water_fraction = particle.max_water_fraction

In [ ]:
# Update Properties

def Update_oil_properties(particle, fieldset, time):
    
    kv1 = math.sqrt(particle.viscosity) * 1.5e3
    if kv1 > 10:
        kv1 = 10
    elif kv1 < 1:
        kv1 = 1
    
    particle.density = particle.water_fraction * fieldset.density_water + (1 - particle.water_fraction) * particle.original_density
    
    particle.viscosity = particle.original_viscosity * math.exp((kv1 * particle.fraction_evaporated) * (1 + (particle.water_fraction/0.84)/(1.187 - (particle.water_fraction/0.84)))**2.49

In [ ]:
# Entrainment

def Entrainment(particle, fieldset, time):
    
    if particle.depth == 0:
        
        d_0 = 4 * math.sqrt(particle.interfacial_tension / (fieldset.g*(fieldset.density_water - particle.density)))
        weber = fieldset.density_water * g * 0.0246 * fieldset.wind_speed**2 * d_0 / particle.interfacial_tension
        ohns = (particle.viscosity * particle.density) / (math.sqrt(particle.density * particle.interfacial_tension * d_0))
        if fieldset.wind_speed > 5:
            T_p = 2*math.pi/(0.877 * fieldset.g * (1/(1.17*fieldset.wind_speed)))
            F_bw = 0.032 * (fieldset.wind_speed - 5)/T_p
        else:
            F_bw = 0
            
        entrainment_rate = 4.604e-10 * weber**1.805 * ohns**(-1.023) * F_bw
        if (entrainment_rate * fieldset.dt) < 0.01:
            entrainment_probability = entrainment_rate * particle.dt
        else:
            entrainment_probability = 1 - math.exp(-1 * entrainment_rate * particle.dt)
        
        if ParcelsRandom.random() < entrainment_probability:
            particle.depth = ParcelsRandom.random() * 1.5 * 0.0246 * fieldset.wind_speed**2
        

In [ ]:
# Turbulent Mixing

def KPP_wind_mixing(particle, fieldset, time):
    """
    Author: Victor Onink, 25/01/22
    """
    # Loading the mixed layer depth from the fieldset
    mld = fieldset.mld

    # Below the MLD there is no wind-driven turbulent diffusion according to KPP theory, so we set both Kz and dKz to zero.
    if particle.depth > mld:
        Kz = 0
        dKz = 0

    # Within the MLD we compute the vertical diffusion according to Boufadel et al. (2020) https://doi.org/10.1029/2019JC015727
    else:
      # Calculate the wind speed at the ocean surface
        w_10 = fieldset.wind_speed

      # Drag coefficient according to Large & Pond (1981) https://doi.org/10.1175/1520-0485(1981)011%3C0324:OOMFMI%3E2.0.CO;2
        C_D = min(max(1.2E-3, 1.0E-3 * (0.49 + 0.065 * w_10)), 2.12E-3)

      # Calculate the surface wind stress based on the surface wind speed and the density of air
        tau = C_D * fieldset.rho_a * w_10 ** 2

      # Calcuate the friction velocity of water at the ocean surface using the surface wind stress and the surface water density
        U_W = math.sqrt(tau / particle.density_water)

      # Calcuate the surface roughness z0 following Zhao & Li (2019) https://doi.org/10.1007/s10872-018-0494-9
        z0 = 3.5153e-5 * fieldset.BETA ** (-0.42) * w_10 ** 2 / fieldset.G

      # The corrected particle depth, since the depth is not always zero for the surface circulation data
        z_correct = particle.z_correct

      # The diffusion gradient at particle.depth
        C1 = (fieldset.vk * U_W * fieldset.theta) / (fieldset.phi * mld ** 2)
        dKz = C1 * (mld - z_correct) * (mld - 3 * z_correct - 2 * z0)

      # The KPP profile vertical diffusion, at a depth corrected for the vertical gradient in Kz
        C2 = (fieldset.vk * U_W * fieldset.theta) / fieldset.phi
        Kz = C2 * (z_correct + z0) * math.pow(1 - z_correct / mld, 2)

    # The Markov-0 vertical transport from Grawe et al. (2012) http://dx.doi.org/10.1007/s10236-012-0523-y
    gradient = dKz * particle.dt
    R = ParcelsRandom.uniform(-1., 1.) * math.sqrt(math.fabs(particle.dt) * 3) * math.sqrt(2 * Kz)

    # Update the particle depth
    particle.depth = particle.depth + gradient + R
    if particle.depth < 0:
        particle.depth = particle.depth * -1

In [ ]:
# Buyonacy velocity rise

def Buoyancy(particle, fieldset, time):
    
    r = ParcelsRandom.uniform(particle.min_diameter, particle.max_diameter)
    particle.rise_velocity = 2 * 9.81 * (1 - (particle.density/fieldset.density_water)) * r**2 / (9 * fieldset.viscosity_water)
    
    if (particle.rise_velocity * 2 * r / fieldset.viscosity_water) > 50:
        particle.rise_velocity = math.sqrt((16/3) * 9.81 * (1 - (particle.density/fieldset.density_water)) * r)
        
    particle.depth = particle.depth - particle.rise_velocity * particle.dt
    
    if particle.depth < 0:
        particle.depth = 0